In [1]:
!pip install -q kaggle
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ifeanyinneji","key":"68b7b1b6c0d102f7d9941d0b3eff800a"}'}

In [2]:
# This cell creates a .kaggle hidden folder in our root directory and copies the kaggle.json file to the folder.
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

In [3]:
# This cell will download the dataset used for this project to the /content directory and also unzip the csv file.
!kaggle datasets download -d darknez/manga-dataset
!unzip /content/manga-dataset.zip

  0% 0.00/4.88M [00:00<?, ?B/s]
100% 4.88M/4.88M [00:00<00:00, 117MB/s]
Archive:  /content/manga-dataset.zip
  inflating: Apr28_2021.csv          
  inflating: Aug2020.csv             
  inflating: May2_2021.csv           
  inflating: May3_2021.csv           


In [4]:
!du -sh *

3.5M	Apr28_2021.csv
5.7M	Aug2020.csv
4.0K	kaggle.json
4.9M	manga-dataset.zip
3.5M	May2_2021.csv
7.0M	May3_2021.csv
55M	sample_data


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
import pandas as pd
df = pd.read_csv("May3_2021.csv")
df2 = pd.read_csv("Aug2020.csv")
df3 = pd.read_csv("May2_2021.csv")
df4 = pd.read_csv("Apr28_2021.csv")

print("Shape", df.shape)
print("Shape", df2.shape)
print("Shape", df3.shape)
print("Shape", df4.shape)

Shape (29665, 12)
Shape (24247, 12)
Shape (14825, 12)
Shape (14819, 12)


In [ ]:
total_genres = []
for g in df3["Genre"]:
    g = g
    if g not in total_genres:
        total_genres.append(g)
total_genres

In [4]:
df.shape

(29665, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29665 entries, 0 to 29664
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              29664 non-null  object
 1   Latest Chapter    29665 non-null  object
 2   Dated Released    29665 non-null  object
 3   depth             29665 non-null  object
 4   download_timeout  29665 non-null  object
 5   download_slot     29665 non-null  object
 6   download_latency  29665 non-null  object
 7   Link              29665 non-null  object
 8   Genre             29280 non-null  object
 9   Status            29332 non-null  object
 10  Rating            29665 non-null  object
 11  img-link          29665 non-null  object
dtypes: object(12)
memory usage: 2.7+ MB


In [8]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df3['Genre'].values.astype('U'))

In [9]:
cos_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

In [22]:
indices=pd.Series(df3.index,index=df3['Name'])
titles=df3['Name']
def recommend_manga(title):
    idx = indices[title]
    similarity_scores = list(enumerate(cos_similarity[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:11]
    data = []
    for i in similarity_scores:
        item = []
        temp_df = df3[df3['Name'] == indices.index[i[0]] ]
        item.extend(list(temp_df.drop_duplicates('Name')['Name'].values ))
        item.extend(list(temp_df.drop_duplicates('Name')['img-link'].values ))
        data.append(item)
    return data

In [23]:
df3.head()

,Name,Latest Chapter,Dated Released,depth,download_timeout,download_slot,download_latency,Link,Genre,Status,Rating,img-link
0,The Eunuch's Consort Rules The World,Chapter 56,"May 02,21",2,180.0,manganelo.com,0.408603,https://manganelo.com/manga/ee923368,"Historical,Romance,Webtoons",Ongoing,4.64,https://avt.mkklcdnv6temp.com/32/w/21-15936633...
1,Remarried Empress,Chapter 77,"May 02,21",2,180.0,manganelo.com,0.476739,https://manganelo.com/manga/gi921111,Romance,Ongoing,4.83,https://avt.mkklcdnv6temp.com/43/x/19-15835007...
2,Dream Life: Yume No Isekai Seikatsu,Chapter 5.1,"May 02,21",2,180.0,manganelo.com,0.614110,https://manganelo.com/manga/ad924426,"Action,Adventure,Comedy,Drama,Fantasy,Mystery,...",Ongoing,4.69,https://avt.mkklcdnv6temp.com/25/h/22-16017844...
3,Hataraku Saibou Black,Chapter 41,"May 02,21",2,180.0,manganelo.com,0.608487,https://manganelo.com/manga/lbza281491570207597,"Action,Drama,School life,Sci fi,Seinen",Ongoing,4.66,https://avt.mkklcdnv6temp.com/44/a/16-15834950...
4,Crush On You,Chapter 71,"May 02,21",2,180.0,manganelo.com,0.665427,https://manganelo.com/manga/crush_on_you,"Comedy,Drama,Josei,Romance,School life,Shoujo,...",Completed,3.43,https://avt.mkklcdnv6temp.com/3/e/4-1583469777...


In [24]:
recommend_manga('Crush On You')

[['Otona No Mondai',
  'https://avt.mkklcdnv6temp.com/35/l/4-1583471145.jpg',
  'https://manganelo.com/manga/otona_no_mondai'],
 ['Girl X Girl X Boy',
  'https://avt.mkklcdnv6temp.com/18/v/5-1583472580.jpg',
  'https://manganelo.com/manga/girl_x_girl_x_boy'],
 ['Become A Girl At Night',
  'https://avt.mkklcdnv6temp.com/42/i/18-1583498741.jpg',
  'https://manganelo.com/manga/er919718'],
 ['The Love Doctor',
  'https://avt.mkklcdnv6temp.com/22/s/14-1583490557.jpg',
  'https://manganelo.com/manga/the_love_doctor'],
 ['Milkman',
  'https://avt.mkklcdnv6temp.com/5/a/9-1583480288.jpg',
  'https://manganelo.com/manga/milkman'],
 ['Ijippari Yuugitai!',
  'https://avt.mkklcdnv6temp.com/12/m/16-1583493923.jpg',
  'https://manganelo.com/manga/ijippari_yuugitai'],
 ['69',
  'https://avt.mkklcdnv6temp.com/9/p/7-1583476656.jpg',
  'https://manganelo.com/manga/69'],
 ['Bingo!',
  'https://avt.mkklcdnv6temp.com/18/c/7-1583476951.jpg',
  'https://manganelo.com/manga/bingo'],
 ['Hirano To Kagiura',
  'h